In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

def run_logistic_regression(file_path):
    try:
        df = pd.read_excel(file_path)

        original_target_col_name_tuple = ('P2_g ', 'Nivel')
        new_column_names = {}
        generated_target_col_name_for_tuple = None

        for col in df.columns:
            if isinstance(col, tuple):
                # Lógica de renomeação original do script anterior para tuplas:
                # 1. Juntar as strings limpas da tupla com underscore
                # 2. Aplicar replaces globais na string resultante
                temp_name_parts = [str(c).strip() for c in col if str(c).strip()]
                new_name = '_'.join(temp_name_parts)
                new_name = new_name.replace("(", "").replace(")", "").replace(" ", "_").replace("/", "_per_").replace("-", "_")
            else:
                new_name = str(col).strip().replace(" ", "_").replace("(", "").replace(")", "").replace("R$", "BRL")

            new_column_names[col] = new_name

            if col == original_target_col_name_tuple:
                generated_target_col_name_for_tuple = new_name

        df = df.rename(columns=new_column_names)

        print("Colunas após renomeação:")
        current_columns = df.columns.tolist()
        print(current_columns)

        target_column = None
        # 1. Tentar usar o nome gerado pela lógica de renomeação para a tupla alvo
        if generated_target_col_name_for_tuple and generated_target_col_name_for_tuple in current_columns:
            target_column = generated_target_col_name_for_tuple
            print(f"Coluna alvo encontrada pelo nome gerado dinamicamente: {target_column}")
        else:
            # 2. Fallback: Se o nome gerado não funcionar, verificar se o nome problemático anterior ('P2_g___Nivel') existe
            # (Isso é improvável de ser o correto, mas para cobrir o erro original)
            # O erro original era KeyError: 'P2_g___Nivel'
            # A lista de colunas impressa era ['ID', "'P1_a_',_'Idade'", ..., "'P2_g_',_'Nivel'", ...]
            # O nome correto gerado pela lógica acima para ('P2_g ', 'Nivel') é 'P2_g_Nivel'

            # Tentativa com o nome que a lógica de renomeação DEVERIA produzir para ('P2_g ', 'Nivel')
            expected_name_from_logic = "P2_g_Nivel"
            if expected_name_from_logic in current_columns:
                target_column = expected_name_from_logic
                print(f"Coluna alvo encontrada pelo nome esperado da lógica de renomeação: {target_column}")
            else:
                # 3. Fallback para o nome que apareceu na impressão de erro anterior, se ele existir
                # A saída do erro anterior mostrava a coluna como "'P2_g_',_'Nivel'"
                # É crucial que esta string seja exatamente como aparece em df.columns.tolist()
                printed_name_candidate = "'P2_g_',_'Nivel'" # String literal como apareceu na lista
                if printed_name_candidate in current_columns:
                    target_column = printed_name_candidate
                    print(f"Coluna alvo encontrada pelo nome exato da lista de colunas impressa anteriormente: {target_column}")
                else:
                    raise ValueError(f"Não foi possível determinar o nome da coluna alvo. \nNome gerado pela lógica: {generated_target_col_name_for_tuple}. \nNome esperado pela lógica: {expected_name_from_logic}. \nCandidato da impressão anterior: {printed_name_candidate}. \nColunas disponíveis: {current_columns}")

        features_columns = [col for col in df.columns if col not in [target_column, 'ID']]
        X = df[features_columns]
        y = df[target_column]

        print(f"\nVariável Alvo selecionada: {target_column}")
        print(f"Valores únicos na variável alvo: {y.unique().tolist()}")
        print(f"Distribuição da variável alvo:\n{y.value_counts(normalize=True).to_string()}")

        numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
        categorical_features = X.select_dtypes(include=['object']).columns.tolist()

        print(f"\nFeatures Numéricas: {numerical_features}")
        print(f"Features Categóricas: {categorical_features}")

        preprocessor = ColumnTransformer(
            transformers=[
                ('num', StandardScaler(), numerical_features),
                ('cat', OneHotEncoder(handle_unknown='ignore', drop='first'), categorical_features)
            ],
            remainder='passthrough'
        )

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
        print(f"\nTamanho do conjunto de treino: {X_train.shape[0]} amostras")
        print(f"Tamanho do conjunto de teste: {X_test.shape[0]} amostras")

        model = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', LogisticRegression(random_state=42, max_iter=1000, multi_class='ovr', solver='lbfgs'))
        ])

        print("\nTreinando o modelo de Regressão Logística...")
        model.fit(X_train, y_train)
        print("Treinamento concluído.")

        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)

        print(f"\nResultados da Regressão Logística:")
        print(f"Acurácia no conjunto de teste: {accuracy:.4f}")
        print("\nRelatório de Classificação:")
        print(report)


    except FileNotFoundError:
        print(f"Erro: O arquivo '{file_path}' não foi encontrado.")
    except Exception as e:
        print(f"Ocorreu um erro ao executar o pipeline de Regressão Logística: {e}")
        import traceback
        print(traceback.format_exc())

if __name__ == "__main__":
    file_path = "Base de dados principal TRATADA.xlsx"
    run_logistic_regression(file_path)


Colunas após renomeação:
['ID', "'P1_a_',_'Idade'", "'P1_b_',_'Genero'", "'P1_c_',_'Cor/raca/etnia'", "'P1_d_',_'PCD'", "'P1_i_',_'Estado_onde_mora'", "'P1_l_',_'Nivel_de_Ensino'", "'P2_g_',_'Nivel'", 'Salário__BRL', 'Tempo_de_experiência_na_área_de_dados_anos', 'IDH_do_Estado']
Coluna alvo encontrada pelo nome exato da lista de colunas impressa anteriormente: 'P2_g_',_'Nivel'

Variável Alvo selecionada: 'P2_g_',_'Nivel'
Valores únicos na variável alvo: ['Sênior', 'Pleno', 'Júnior']
Distribuição da variável alvo:
'P2_g_',_'Nivel'
Pleno     0.370981
Sênior    0.348997
Júnior    0.280022

Features Numéricas: ["'P1_a_',_'Idade'", 'Salário__BRL', 'Tempo_de_experiência_na_área_de_dados_anos', 'IDH_do_Estado']
Features Categóricas: ["'P1_b_',_'Genero'", "'P1_c_',_'Cor/raca/etnia'", "'P1_d_',_'PCD'", "'P1_i_',_'Estado_onde_mora'", "'P1_l_',_'Nivel_de_Ensino'"]

Tamanho do conjunto de treino: 2911 amostras
Tamanho do conjunto de teste: 728 amostras

Treinando o modelo de Regressão Logística...

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
